In [67]:
import pypdfium2 as pdfium
import re
import pandas as pd
import pickle
import openai
import time
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [68]:
pdf = pdfium.PdfDocument("../02465_Notes.pdf")

text_all = ""
for page in pdf:
    textpage = page.get_textpage()
    text_all += " ".join(textpage.get_text_range().splitlines())

In [69]:
text_sentences = list(filter(None, re.split('\. |\? |\! ', text_all)))
text_sentences = [seg for seg in text_sentences if len(seg)>10]

In [70]:
text_segments = []
seg = ""
threshold = 1000 #chars
for s in text_sentences:
    if len(seg)+len(s)>threshold:
        text_segments.append(seg)
        seg = ""
    else:
        seg+=f' {s}'

text_segments = [seg for seg in text_segments if len(seg)>10]

In [71]:
len(text_segments)

371

In [72]:
# Ask ChatGPT to make questions to sections
combinations = []

req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
for i in range(0, len(text_segments)):
    context = text_segments[i]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 5 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 5 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 5:
        print(f'[Error] questions not generated for {i}')
        continue
    
    # Save data into a dataframe
    combinations += [{'context': context, 'question': question} for question in questions]
    print(f"Questions from round {i+1} saved!")

Questions from round 1 saved!
Questions from round 2 saved!
Questions from round 3 saved!
Questions from round 4 saved!
Questions from round 5 saved!
Questions from round 6 saved!
Questions from round 7 saved!
Questions from round 8 saved!
Questions from round 9 saved!
Questions from round 10 saved!
Questions from round 11 saved!
Questions from round 12 saved!
Questions from round 13 saved!
Questions from round 14 saved!
Questions from round 15 saved!
Questions from round 16 saved!
Questions from round 17 saved!
Questions from round 18 saved!
Questions from round 19 saved!
Questions from round 20 saved!
Questions from round 21 saved!
Questions from round 22 saved!
Questions from round 23 saved!
Questions from round 24 saved!
Questions from round 25 saved!
Questions from round 26 saved!
Questions from round 27 saved!
Questions from round 28 saved!
Questions from round 29 saved!
Questions from round 30 saved!
Questions from round 31 saved!
Questions from round 32 saved!
Questions from ro

In [73]:
df = pd.DataFrame(columns=['context', 'question'], data = combinations)

with open('./data/tue_rl.pkl', 'wb') as fp:
    pickle.dump(df, fp)

In [74]:
df

,context,question
0,LECTURE NOTES 02465 Sequential Decision-Makin...,"What is the purpose of the course 02450, Intro..."
1,LECTURE NOTES 02465 Sequential Decision-Makin...,What is the underlying problem that both contr...
2,LECTURE NOTES 02465 Sequential Decision-Makin...,Why is it not possible to find a unified treat...
3,LECTURE NOTES 02465 Sequential Decision-Makin...,What resource is used for the second half of t...
4,LECTURE NOTES 02465 Sequential Decision-Makin...,What difficulties did the author face in findi...
...,...,...
1850,Metropolis and S The monte carlo method Assoc...,What is the title of the book written by Stuar...
1851,Metropolis and S The monte carlo method Assoc...,In which year was the Monte Carlo method first...
1852,Metropolis and S The monte carlo method Assoc...,"Who are the authors of the paper titled ""Learn..."
1853,Metropolis and S The monte carlo method Assoc...,"What is the focus of the paper titled ""Data-dr..."
